In [ ]:
# | default_exp _components.aiokafka_loop

In [ ]:
# | export
from typing import *

from os import environ
import asyncio

from aiokafka import AIOKafkaConsumer
from pydantic import BaseModel, HttpUrl, NonNegativeInt, Field
import asyncer
import anyio

from fast_kafka_api.logger import get_logger, supress_timestamps
from fast_kafka_api.testing import true_after, create_and_fill_testing_topic, nb_safe_seed
from fast_kafka_api.asyncapi import KafkaMessage

[INFO] fast_kafka_api.asyncapi: ok


In [ ]:
seed = nb_safe_seed("_components.aiokafka_loop")

In [ ]:
# | eval: false
# allows async calls in notebooks

import nest_asyncio
nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.debug("ok")

In [ ]:
kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {
    "bootstrap.servers": f"{kafka_server_url}:{kafka_server_port}"
}

In [ ]:
class MyMessage(BaseModel):
    url: HttpUrl = Field(..., example="http://www.acme.com", description="Url example")
    port: NonNegativeInt = Field(1000)

In [ ]:
async def process_items(receive_stream):
    async with receive_stream:
        async for callback, msg, produce in receive_stream:
            await callback(msg, produce)

In [ ]:


async def aiokafka_consumer_loop(
    topics: List[str],
    *,
    bootstrap_servers: str,
    auto_offset_reset: str,
    max_poll_records: int,
    callbacks: Dict[
        str, Callable[[KafkaMessage, Callable[[str, KafkaMessage], None]], None]
    ],
    produce: Callable[[str, KafkaMessage], None],
    msg_types: Dict[str, Type[KafkaMessage]],
    is_shutting_down_f: Callable[[], bool],
):
    consumer = AIOKafkaConsumer(
        bootstrap_servers=bootstrap_servers,
        auto_offset_reset=auto_offset_reset,
        max_poll_records=max_poll_records,
    )
    logger.info("Consumer created.")

    await consumer.start()
    logger.info("Consumer started.")
    consumer.subscribe(topics)
    logger.info("Consumer subscribed.")
    
    msgs_received = 0
    try:
        send_stream, receive_stream = anyio.create_memory_object_stream()
        async with anyio.create_task_group() as tg:
            tg.start_soon(process_items, receive_stream)
            async with send_stream:
                while True:
                    msgs = await consumer.getmany(timeout_ms=100)
                    for topic_partition, msgs in msgs.items():
                        topic = topic_partition.topic
                        msg_type = msg_types[topic]
                        decoded_msgs = [
                            msg_type.parse_raw(msg.value.decode("utf-8")) for msg in msgs
                        ]
                        for msg in decoded_msgs:
                            await send_stream.send((callbacks[topic], msg, produce))
                        msgs_received = msgs_received + len(msgs)
                    if is_shutting_down_f():
                        break

    finally:
        await consumer.stop()
        logger.info(f"Consumer stopped.")

In [ ]:
msgs = [
    MyMessage(url="http://www.ai.com", port=port).json().encode("utf-8")
    for port in range(1000, 10000)
]
msgs_sent = len(msgs)

async def print_msg(msg: MyMessage, produce):
    if msg.port % 100 == 0:
        print(msg)
        await produce("my_topic", msg)
    
async def produce_print_msg(topic: str, msg: MyMessage):
    print(f"Producing {msg} for {topic}")

async with create_and_fill_testing_topic(kafka_config=kafka_config, msgs=msgs, seed=seed(14)) as topic:
    await aiokafka_consumer_loop(
        topics = [topic],
        bootstrap_servers = kafka_config["bootstrap.servers"],
        auto_offset_reset="earliest",
        max_poll_records=100,
        callbacks = {topic: print_msg},
        produce = produce_print_msg,
        msg_types= {topic: MyMessage},
        is_shutting_down_f= true_after(5),
    )

[INFO] fast_kafka_api.testing: create_missing_topics(['my_topic_2774042165']): new_topics = [NewTopic(topic=my_topic_2774042165,num_partitions=3)]
[INFO] fast_kafka_api.testing: Producer <aiokafka.producer.producer.AIOKafkaProducer object> created.
[INFO] fast_kafka_api.testing: Producer <aiokafka.producer.producer.AIOKafkaProducer object> stared.
[INFO] fast_kafka_api.testing: Sent messages: len(sent_msgs)=9000
[INFO] __main__: Consumer created.
[INFO] __main__: Consumer started.
[INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'my_topic_2774042165'})
[INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'my_topic_2774042165'}
[INFO] __main__: Consumer subscribed.
[INFO] aiokafka.consumer.group_coordinator: Metadata for topic has changed from {} to {'my_topic_2774042165': 3}. 
url=HttpUrl('http://www.ai.com', ) port=1000
Producing url=HttpUrl('http://www.ai.com', ) port=1000 for my_topic
url=HttpUrl('http://www.ai.com', ) port=1400
Producing

url=HttpUrl('http://www.ai.com', ) port=6300
Producing url=HttpUrl('http://www.ai.com', ) port=6300 for my_topic
url=HttpUrl('http://www.ai.com', ) port=7100
Producing url=HttpUrl('http://www.ai.com', ) port=7100 for my_topic
url=HttpUrl('http://www.ai.com', ) port=7200
Producing url=HttpUrl('http://www.ai.com', ) port=7200 for my_topic
url=HttpUrl('http://www.ai.com', ) port=7300
Producing url=HttpUrl('http://www.ai.com', ) port=7300 for my_topic
url=HttpUrl('http://www.ai.com', ) port=7500
Producing url=HttpUrl('http://www.ai.com', ) port=7500 for my_topic
url=HttpUrl('http://www.ai.com', ) port=7600
Producing url=HttpUrl('http://www.ai.com', ) port=7600 for my_topic
url=HttpUrl('http://www.ai.com', ) port=8000
Producing url=HttpUrl('http://www.ai.com', ) port=8000 for my_topic
url=HttpUrl('http://www.ai.com', ) port=8800
Producing url=HttpUrl('http://www.ai.com', ) port=8800 for my_topic
url=HttpUrl('http://www.ai.com', ) port=8900
Producing url=HttpUrl('http://www.ai.com', ) port=8